<a href="https://colab.research.google.com/github/danielsoy/ALOCC-CVPR2018/blob/master/PADIM%20ANODET%202025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Padim Example

#### Import dependencies

In [1]:
!git clone https://github.com/OpenAOI/anodet.git

Cloning into 'anodet'...
remote: Enumerating objects: 646, done.
remote: Counting objects: 100% (227/227), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 646 (delta 150), reused 166 (delta 118), pack-reused 419 (from 1)
Receiving objects: 100% (646/646), 10.20 MiB | 17.19 MiB/s, done.
Resolving deltas: 100% (355/355), done.


In [2]:
%cd /content/anodet/

/content/anodet


In [3]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
!pip install anodet

ERROR: Could not find a version that satisfies the requirement anodet (from versions: none)
ERROR: No matching distribution found for anodet


In [5]:
import os
import anodet
import numpy as np
import torch
import cv2
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from time import sleep


#

## Training

In this notebook the MVTec dataset will be used. It can be downloaded from: https://www.mvtec.com/company/research/datasets/mvtec-ad.

Change `DATASET_PATH` to the path where you save the dataset.

Also, if you want to save and load the model. Add a folder where to save them and change the path of `MODEL_DATA_PATH` to that folder.

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
DATASET_PATH = os.path.realpath("/content/drive/MyDrive/bottle")
MODEL_DATA_PATH = os.path.realpath("/content/modelos")

In [8]:
dataset = anodet.AnodetDataset(os.path.join(DATASET_PATH, "/content/drive/MyDrive/bottle/train/good"))
dataloader = DataLoader(dataset, batch_size=32)
print("Number of images in dataset:", len(dataloader.dataset))

Number of images in dataset: 209


#### Init the model

In [9]:
padim = anodet.Padim(backbone='resnet18')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 73.3MB/s]


#### Fit the model to the dataset

In [10]:
padim.fit(dataloader)

Feature extraction: 100%|██████████| 7/7 [00:49<00:00,  7.06s/it]


#### Save the necessary parameters

In [11]:
distributions_path = './distributions/'
torch.save(padim.mean, os.path.join(MODEL_DATA_PATH, "bottle_mean.pt"))
torch.save(padim.cov_inv, os.path.join(MODEL_DATA_PATH, "bottle_cov_inv.pt"))

RuntimeError: Parent directory /content/modelos does not exist.

In [ ]:
cap=cv2.VideoCapture(0)  # si no se ve nada, ver si la camara esta funcionando

#

## Inference
#### Load test images

In [ ]:
mean = torch.load(os.path.join(MODEL_DATA_PATH, 'bottle_mean.pt'))
cov_inv = torch.load(os.path.join(MODEL_DATA_PATH, 'bottle_cov_inv.pt'))

In [ ]:
while True:
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  ### +COLOR +luz ,+sensibilidad. mas traininig, mas sutiliza.
    frame = cv2.circle(frame, (320, 240), 350, (0,0,0), thickness=240 , lineType=cv2.LINE_AA)  # mascara original externa
    cv2.imwrite("C:\\Users\libro/anodet\mvtec/anodet/webcam.jpg",frame)



    paths = [os.path.join(DATASET_PATH, "anodet/webcam.jpg"),
    ]

    images = []
    for path in paths:
        image = cv2.imread(path)
       # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        images.append(image)

    batch = anodet.to_batch(images, anodet.standard_image_transform, torch.device('cpu'))


    # mean = torch.load(os.path.join(MODEL_DATA_PATH, 'bottle_mean.pt'))
    # cov_inv = torch.load(os.path.join(MODEL_DATA_PATH, 'bottle_cov_inv.pt'))


    padim = anodet.Padim(backbone='resnet18', mean=mean, cov_inv=cov_inv, device=torch.device('cpu'))


    image_scores, score_maps = padim.predict(batch)


    THRESH = 19
    score_map_classifications = anodet.classification(score_maps, THRESH)
    image_classifications = anodet.classification(image_scores, THRESH)
    print("Image scores:", image_scores)
    print("Image classifications:", image_classifications)



    test_images = np.array(images).copy()


    boundary_images = anodet.visualization.framed_boundary_images(test_images, score_map_classifications, image_classifications, padding=40)
    heatmap_images = anodet.visualization.heatmap_images(test_images, score_maps, alpha=0.5)
    highlighted_images = anodet.visualization.highlighted_images(images, score_map_classifications, color=(128, 0, 128))

    for idx in range(len(images)):
        fig, axs = plt.subplots(1, 4, figsize=(12, 6))
        fig.suptitle('Image: ' + str(idx), y=0.75, fontsize=14)
        axs[0].imshow(images[idx])
        axs[1].imshow(boundary_images[idx])
        axs[2].imshow(heatmap_images[idx])
        axs[3].imshow(highlighted_images[idx])
        #plt.show()


        plt.show(block=False)
        plt.pause(5)
        plt.close()


        print("fin")

In [ ]:
qqqqqqqqqqqqqqqqqqq